In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [2]:
# CHANGE ME!
TASK_ID = 'exists_outlier'
AUTHOR = 'g'


In [22]:
task = Task(
    id=TASK_ID,
    name= "Unique product",
    description="Find if there is a product that is different from all other products on a shelf",
    plan=[
    ],
    sample_input={
       "list":[{"id":0,"value":"Apple"},{"id":1,"value":"Banana"},{"id":2,"value":"Banana"},
               {"id":3,"value":"Banana"} ]
    },
    sample_output=1,
    category='Basic'
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [23]:
imp = replace(prototype,
    language='python-imperative',
    source='''
def exists_unique(list):
    values = [l["value"] for l in list]
    for elem in values:
        if values.count(elem) == 1:
            return 1
    return 0
''')
imp.execute(task)
imp.save()

In [24]:
func = replace(prototype,
    language='python-functional',
    source='''
def exists_unique(list):
    values = [l["value"] for l in list]
    return int(any(values.count(x) == 1 for x in values))
''')
func.execute(task)
func.save()

In [25]:
pandas = replace(prototype,
    language='python-pandas',
    source='''
def exists_unique(list):
    values = list.value.value_counts()
    for val in values:
        if val == 1:
            return 1
    return 0
''')
pandas.execute(task)
pandas.save()

In [38]:
func = replace(prototype,
    language='sql',
    source='''
    SELECT CASE WHEN EXISTS (
    SELECT value
    FROM list
    GROUP BY value
    HAVING Count(*) == 1
    )
    THEN CAST(1 AS INT)
    ELSE CAST(0 AS INT)
    END
''')
func.execute(task)
func.save()

In [44]:
datalog = replace(prototype,
    language='datalog',
    source='''
.decl is_unique(value:number)
is_unique(value) :-
    list(_,value), !not_unique(value).
    
.decl not_unique(value:number)
not_unique(value) :- 
    list(id1,value), list(id2,value), id1 != id2.

exists_unique(1) :- is_unique(_).
exists_unique(0) :- is_unique(_).

''').load_plan()
datalog.execute(task)
datalog.save()

Error: Unable to deduce type for variable value in file program.dl at line 7
is_unique(value) :-
----------^---------
Error: Unable to deduce type for variable value in file program.dl at line 8
    list(_,value), !not_unique(value).
-----------^---------------------------
Error: Unable to deduce type for variable value in file program.dl at line 8
    list(_,value), !not_unique(value).
-------------------------------^-------
Error: Unable to deduce type for variable value in file program.dl at line 11
not_unique(value) :- 
-----------^----------
Error: Unable to deduce type for variable value in file program.dl at line 12
    list(id1,value), list(id2,value), id1 != id2.
-------------^------------------------------------
Error: Unable to deduce type for variable value in file program.dl at line 12
    list(id1,value), list(id2,value), id1 != id2.
------------------------------^-------------------
6 errors generated, evaluation aborted



CalledProcessError: Command 'souffle -F. -D. program.dl' returned non-zero exit status 1.